# PySpark explain()

In [2]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import quinn
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 12:33:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.createDataFrame([("li", 3), ("chang", 58), ("carlos", 34)]).toDF("first_name", "age")

In [11]:
df.show()

+----------+---+
|first_name|age|
+----------+---+
|        li|  3|
|     chang| 58|
|    carlos| 34|
+----------+---+



In [6]:
from pyspark.sql.functions import col
df.where(col("age") > 30).show()

+----------+---+
|first_name|age|
+----------+---+
|     chang| 58|
|    carlos| 34|
+----------+---+



In [8]:
df.where(col("age") > 30).explain()

== Physical Plan ==
*(1) Project [_1#0 AS first_name#4, _2#1L AS age#5L]
+- *(1) Filter (isnotnull(_2#1L) AND (_2#1L > 30))
   +- *(1) Scan ExistingRDD[_1#0,_2#1L]




In [9]:
spark.sql("select * from {df} where age > 30", df=df).show()

+----------+---+
|first_name|age|
+----------+---+
|     chang| 58|
|    carlos| 34|
+----------+---+



In [10]:
spark.sql("select * from {df} where age > 30", df=df).explain()

== Physical Plan ==
*(1) Project [_1#0 AS first_name#4, _2#1L AS age#5L]
+- *(1) Filter (isnotnull(_2#1L) AND (_2#1L > 30))
   +- *(1) Scan ExistingRDD[_1#0,_2#1L]




In [12]:
spark.sql("select * from {df} where age > 30", df=df).explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('age > 30)
   +- 'UnresolvedRelation [_pyspark_652a30c77ff64c31b9452248661ac88d], [], false

== Analyzed Logical Plan ==
first_name: string, age: bigint
Project [first_name#4, age#5L]
+- Filter (age#5L > cast(30 as bigint))
   +- SubqueryAlias _pyspark_652a30c77ff64c31b9452248661ac88d
      +- View (`_pyspark_652a30c77ff64c31b9452248661ac88d`, [first_name#4,age#5L])
         +- Project [_1#0 AS first_name#4, _2#1L AS age#5L]
            +- LogicalRDD [_1#0, _2#1L], false

== Optimized Logical Plan ==
Project [_1#0 AS first_name#4, _2#1L AS age#5L]
+- Filter (isnotnull(_2#1L) AND (_2#1L > 30))
   +- LogicalRDD [_1#0, _2#1L], false

== Physical Plan ==
*(1) Project [_1#0 AS first_name#4, _2#1L AS age#5L]
+- *(1) Filter (isnotnull(_2#1L) AND (_2#1L > 30))
   +- *(1) Scan ExistingRDD[_1#0,_2#1L]

